## __missing__ 을 사용해 키에 따라 다른 디폴트 값을 생성하는 방법을 알아두라

예를 들어 파일 시스템에 있는 SNS 프로필 사진을 관리하는 프로그램을 작성한다고 가정하자.
- 필요할 때 파일을 읽고 쓰기 위해 프로필 사진의 경로와 열린 파일 핸들을 연관시켜주는 딕셔너리가 필요하다.

In [1]:
pictures = {}
path = 'profile_1234.png'

if (handle := pictures.get(path)) is None:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f"경로를 열 수 없습니다.: {path}")
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()

In [ ]:
# 문제가 많은 코드...
# 내장 함수인 open이 딕셔너리에 경로가 있는지 여부와 관계없이 항상 호출된다.
# 같은 프로그램상에 존재하던 열린 파일 핸들과 혼동될 수 있는 새로운 파일 핸들이 생길 수도 있다.
# open이 예외를 던질 수 있으므로 이 예외를 처리해야 한다. 하지만 이 예외를 같은 줄에 있는 setdefault 가 던지는 예외와 구분하지 못할 수도 있다.

try:
    handle = pictures.setdefault(path, open(path, 'a+b'))
except OSError:
    print(f"경로를 열 수 없습니다.: {path}")
    raise
else:
    handle.seek(0)
    image_data = handle.read()

In [4]:
# 이것의 문제는 생성자에 전달한 함수는 인자를 받을 수 없다는 데 있다.
# defaultdict 가 호출하는 도우미 함수가 처리 중인 키를 알 수 없다는 뜻이다.
# 이로 인해 파일 경로를 사용해 open 을 호출할 방법이 없다.


from collections import defaultdict

def open_pictures(profile_path):
    try:
        return open(profile_path, 'a+b')
    except OSError:
        print(f"경로를 열 수 없습니다.: {path}")
        raise

pictures = defaultdict(open_pictures)
handle = pictures[path]
handle.seek(0)
image_data = handle.read()

TypeError: open_pictures() missing 1 required positional argument: 'profile_path'

`__missing__` 을 써서 키가 없는 경우를 처리하는 로직을 커스텀화할 수 있다.

In [5]:
class Pictures(dict):
    def __missing__(self, key):
        value = open_pictures(key)
        self[key] = value
        
        return value
    
pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()